In [ ]:
# Training example
# 
# Using the match game training_data.tgz as created by running scripts/download_match_games/download_match_games.py
# followed by scripts/download_match_games/make_training_data.py
#
# TODO: Need to shuffle data, add a test/train split.

import pickle
import tarfile
import chess.pgn
import io
from lcztools import LeelaBoard, load_network
from torch import optim, nn
import numpy as np
import torch

net = load_network()

for name, param in net.model.named_parameters():
    # print(name)
    if ('_val' in name) and not ('conv1_bn.weight' in name):
        print('+', name)
        param.requires_grad = True
    else:
        # print('-', name)
        param.requires_grad = False

optimizer = optim.Adam(net.model.parameters(), lr = 0.0002, weight_decay=1e-5)
criterion = nn.MSELoss()

losses = []

def do_step(training_game):
    if training_game[1] == '1/2-1/2':
        result = 0
    elif training_game[1] == '1-0':
        result = 1
    elif training_game[1] == '0-1':
        result = -1
    elif training_game[1] == '*':
        return
    features_stack = []
    results = []
    for compressed_features in training_game[0]:
        features_stack.append(LeelaBoard.decompress_features(compressed_features))
        results.append(result)
        result *= -1
    features_stack = np.stack(features_stack)
    
    optimizer.zero_grad()
    pols, vals = net.model(features_stack)
    results = torch.Tensor(results).view(-1, 1).cuda()
    loss = criterion(vals, results)
    losses.append(loss.item())
    if len(losses)==100:
        print(sum(losses)/len(losses))
        losses.clear()
    loss.backward()
    optimizer.step()    

with tarfile.open('training_data.tgz') as f:
    for idx, member in enumerate(f):
        if member.isfile():
            training_game = pickle.load(f.extractfile(member))
            do_step(training_game)

net.model.save_weights_file('/home/trevor/projects/lczero/weights/txt/test_weights_3.txt')

Loading network using backend=pytorch_train_cuda, policy_softmax_temp=2.2
Channels 256
Blocks 20
Enabling CUDA!
+ conv_val.conv1.weight
+ conv_val.conv1_bn.bias
+ affine_val_1.weight
+ affine_val_1.bias
+ affine_val_2.weight
+ affine_val_2.bias
0.33122138580307364
0.32327708806842564
0.342647999804467
0.30650920746847987
0.34121590862050655
0.31692144978791476
0.2926337251719087
0.3612888185679913
0.32078073009848596
0.33477764517068864
0.33724299166351557
0.29047099124640224
0.3376015657559037
0.3247099100984633
0.3170356016419828
0.2968645826354623
0.3251196685619652
0.33669298788532614
0.26937713043764233
0.33238239696249366
0.31816340910270813
0.32765887467190624
0.3277877380698919
0.3299694628827274
0.33974271538667383
0.33874722172040495
0.28303455530665816
0.29582444167695937
0.3109586057346314
0.31915072560776026
0.26685509014409037
0.31726893690880387
0.28230360097251833
0.2911954136285931
0.31076860865578054
0.3014677645917982
0.33730487290304156
0.25808153552934526
0.3813514